<h1 align="center"><a href="https://github.com/sborquez/volcano-seismic_classifier">Volcano-Seismic Classifier</a> - Automatic classification of seismic signals from Llaima volcano (Chile).</h1>

<center>
<br>
<img align="center" src="images/utfsm.png" width="50%"/>

<h2 align="center">K-Fold Cross-Validation</h2>
</center>

<center>
<i> Notebook created by Sebastián Bórquez G. - <a href="mailto://sebstian.borquez@sansano.usm.cl">sebastian.borquez@sansano.usm.cl</a> - utfsm - Jul 2021.</i>
</center>


In [7]:
import data
import models
import performance

from tools import *

In [8]:
!pip install wandb --upgrade -qqq

In [9]:
#os.environ['WANDB_MODE'] = 'online'

# Table of Content

* [K-Fold Cross-Validation](#crossvalidation)
* [SeismicNet](#seismic)
* [Multi-Resolution Convolutional Neural Network](#mrcnn)
* [Time Series Transformer](#transformer)
* [References](#references)

<a id="crossvalidation"><a/>

# K-Fold Cross-Validation

![Kfold](images/02/cross_validation.png)



## Model Training

In [13]:
def training_fold(sweep_q, worker_q):
    worker_data = worker_q.get()
    run_name = "{}-{}".format(worker_data.sweep_run_name, worker_data.num)
    config = worker_data.config
    dataset_artifact_name = worker_data.dataset_artifact_name
    with wandb.init(project="volcano-seismic", group=worker_data.sweep_id, job_type="model-training", name=run_name,
                    config=config,tags=["test"]) as run:    
        # Model builder configuration
        ########################################
        model_hyperparameters = config["model_hyperparameters"]
        training_configuration = config["training_configuration"]
        evaluation_configuration = config["evaluation_configuration"]
        builder = models.builders.get(model_hyperparameters["builder"])
        hyper_parameters = dict(model_hyperparameters)
        del hyper_parameters["builder"]
        # Training configuration
        ########################################
        epochs = training_configuration["epochs"]
        loss =  models.get_loss(**training_configuration["loss"])
        optimizer =  models.get_optimizer(**training_configuration["optimizer"])
        batch_size = training_configuration["batch_size"]
        verbosity = training_configuration["verbosity"]
        # Load Data
        ########################################
        labels = worker_data.labels
        num_classes = worker_data.num_classes
        # ✔️ declare which artifact we'll be using
        dataset_artifact = run.use_artifact(dataset_artifact_name)
        # 📥 if need be, download the artifact
        dataset = dataset_artifact.download()
        X,y = data.read_dataset(dataset, name="processed")
        # Split folds
        train_index = sklearn.utils.shuffle(worker_data.train_index) # I don't know why is not shuffle
        X_train, y_train = X[train_index], y[train_index]
        test_index = worker_data.test_index
        X_test, y_test = X[test_index], y[test_index]
        print('Num clases:', num_classes)
        print('Labels:', labels, type(labels))
        print('X_train shape:', X_train.shape)
        print('y_train shape:', y_train.shape)
        print('X_test shape:', X_test.shape)
        print('y_test shape:', y_test.shape)
        # Load Model
        ########################################
        model = builder(**hyper_parameters)
        # Compile the model
        model.compile(
            loss=loss,
            optimizer=optimizer,
            metrics=['categorical_accuracy' if num_classes > 2 else 'binary_accuracy']
        )
        assert model.predict(2*np.random.random((batch_size,6000,1)) - 1).shape == (batch_size, num_classes), "Incorrect output shape."
        # Train fold
        ########################################
        print(f'Training for fold {worker_data.num+1} ...')
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                  epochs=epochs, verbose=verbosity, batch_size=batch_size,
                  callbacks=[
                      wandb.keras.WandbCallback(verbose=verbosity, labels=labels, save_weights_only=True),
                      tf.keras.callbacks.EarlyStopping(patience=5)
                  ]
        )
        # Evaluation
        ########################################
        # Evaluation configuration
        metrics = evaluation_configuration["metrics"]
        plots = evaluation_configuration["plots"]
        top_k = evaluation_configuration["top_k"]
        # load best mode
        best_model = model
        best_model.load_weights(Path(run.dir)/"model-best.h5")
        y_prob = best_model.predict(X_test)
        y_test = y_test.argmax(axis=-1) if num_classes > 2 else y_test
        model_metrics, model_metrics_tables = performance.get_metrics(y_test, y_prob, metrics=metrics, labels=labels, num_classes=num_classes, run_id=worker_data.num)
        model_plots = performance.get_plots(X_test, y_test, y_prob, plots=plots, labels=labels, num_classes=num_classes)
        model_highlights= performance.get_highlights(X_test, y_test, y_prob, test_index, labels, top_k=top_k)
        # Log to WandB
        ########################################
        run.log(
            dict(
                **model_metrics,
                **model_metrics_tables,
                **model_plots,
                **model_highlights
            )
        )
        sweep_q.put(WorkerDoneData(
            metrics=model_metrics,
            X_test=X_test, y_test=y_test, y_prob=y_prob
        ))
        run.join()
    print(f"Worker {worker_data.num} finished")
    

def k_fold_cross_validation(model_hyperparameters, training_configuration, evaluation_configuration, k=10, seed=None):
    # K-fold Parameters
    ########################################
    num_folds = k
    dataset_artifact_name = 'llaima-preprocessed-1d:latest'
    labels = ["lp", "tc", "tr", "vt"] 
    num_classes = len(labels)
    # Spin up workers before calling wandb.init()
    ########################################
    # Workers will be blocked on a queue waiting to start
    sweep_q = multiprocessing.Queue()
    workers = []
    for num in range(num_folds):
        q = multiprocessing.Queue()
        p = multiprocessing.Process(
            target=training_fold, kwargs=dict(sweep_q=sweep_q, worker_q=q)
        )
        p.start()
        workers.append(Worker(queue=q, process=p))
    # Setup Cross-Validation WandB Runs Group
    ##########################################
    config = {
        "model_hyperparameters": model_hyperparameters,
        "training_configuration": training_configuration,
        "evaluation_configuration": evaluation_configuration,
        "seed": seed
    }
    #sweep_run = wandb.init(project="volcano-seismic", job_type="cross-validation", config=config,tags=["test"])
    with wandb.init(project="volcano-seismic", job_type="cross-validation", config=config) as sweep_run:
        sweep_id = sweep_run.id
        project_url = sweep_run.get_project_url()
        sweep_group_url = "{}/groups/{}".format(project_url, sweep_id)
        sweep_run.notes = sweep_group_url
        sweep_run.save()
        sweep_run_name = sweep_run.name #or sweep_run.id or "unknown"
        print("*" * 40)
        print("Sweep Group URL: ", sweep_group_url)
        print("*" * 40)  
        # Load Data
        ########################################
        # declare which artifact we'll be using
        dataset_artifact = sweep_run.use_artifact(dataset_artifact_name)
        # 📥 if need be, download the artifact
        dataset = dataset_artifact.download()
        X,y = data.read_dataset(dataset, name="processed")
        print('Num clases:', num_classes)
        print('Labels:', labels)
        print('X shape:', X.shape)
        print('y shape:', y.shape)
        # Split 10-Folds
        ########################################
        # Define the K-fold Cross Validator
        evaluations = []
        kfold = sklearn.model_selection.StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=seed)
        # K-fold Cross Validation model evaluation
        for num, (train_index, test_index) in enumerate(kfold.split(X, y.argmax(axis=-1))):
            print('------------------------------------------------------------------------')
            print(f'Worker for fold {num+1} ...')
            config = {
                "model_hyperparameters" : model_hyperparameters,
                "training_configuration": training_configuration,
                "evaluation_configuration": evaluation_configuration,
                "fold": num
            }
            config.update(sweep_run.config)
            # start worker
            worker = workers[num]
            worker.queue.put(
                WorkerInitData(
                    sweep_id=sweep_id,
                    num=num,
                    sweep_run_name=sweep_run_name,
                    config=config,
                    dataset_artifact_name=dataset_artifact_name,
                    train_index=train_index,
                    test_index=test_index,
                    labels=labels,
                    num_classes=num_classes
                )
            )
            #get metric from worker
            result = sweep_q.get()
            # log metric to sweep_run
            evaluations.append(result.metrics)
        # Table of fold metrics and mean/variance row
        ########################################
        # folds
        metrics_df = pd.DataFrame(evaluations).reset_index().rename(columns={"index": "id"})
        metrics_df["id"] = metrics_df["id"].apply(lambda f: f'fold {f}')
        # mean
        metrics_mean = metrics_df.mean()
        metrics_mean["id"] = "mean"
        metrics_df = metrics_df.append(metrics_mean, ignore_index=True)
        # std
        metrics_std = metrics_df.std()
        metrics_std["id"] = "std"
        metrics_df = metrics_df.append(metrics_std, ignore_index=True)
        # Log Table, mean and variance of all metrics
        ########################################
        metrics_mean_dict = {f"mean {m}": v for m,v in metrics_mean.drop(labels=["id"]).iteritems()}
        metrics_std_dict = {f"std {m}": v for m,v in metrics_mean.drop(labels=["id"]).iteritems()}
        ## Log to WandB
        sweep_run.log(dict(
            evaluation = wandb.Table(dataframe=metrics_df),
            **metrics_mean_dict,
            **metrics_std_dict,
        ))
        # Wait for worker to finish 
        ########################################
        #sweep_run.join()
        for num in range(num_folds):
            print("wainting", num)
            worker = workers[num]
            worker.process.join()
    return metrics_df

<a id="SeismicNet"><a/>

# SeismicNet

In [14]:
model_hyperparameters = {
    "builder": "mlp_builder",
    "input_shape": (6000,1),
    "num_classes":4,
    "layers_units": [256, 128]
}

training_configuration = {
    "epochs": 2,
    "batch_size": 128,
    "verbosity": 0,
    "loss": {
        "name": "categorical_crossentropy",
        "parameters": {}
    },
    "optimizer": {
        "name": "adam",
        "parameters": {
            "learning_rate": 0.001
        }
    },
        
}
evaluation_configuration = {
    "metrics": ["overall_classification_metrics", "classification_metrics_by_class"],
    "plots": ["pr", "cm"],
    "top_k": 10
}

In [ ]:
evaluation_metrics = k_fold_cross_validation(model_hyperparameters, training_configuration, evaluation_configuration, k=10, seed=1234)

In [18]:
evaluation_metrics

,id,ACC Macro,ERR Macro,PPV Macro,TNR Macro,TPR Macro,F1 Macro,Kappa,TP - lp,TP - tc,...,TPR - tr,TPR - vt,ACC - lp,ACC - tc,ACC - tr,ACC - vt,F1 - lp,F1 - tc,F1 - tr,F1 - vt
0,fold 0,0.846047,0.153953,0.0,0.875134,0.455930,0.411645,0.499641,643.0,587.0,...,0.053061,0.0,0.803452,0.796214,0.869154,9.153675e-01,0.784625,0.762338,0.099617,0.0
1,fold 1,0.808463,0.191537,0.0,0.859604,0.475903,0.435525,0.422564,650.0,350.0,...,0.440816,0.0,0.787862,0.724388,0.806236,9.153675e-01,0.773349,0.585774,0.382979,0.0
2,mean,0.827255,0.172745,0.0,0.867369,0.465916,0.423585,0.461103,646.5,468.5,...,0.246939,0.0,0.795657,0.760301,0.837695,9.153675e-01,0.778987,0.674056,0.241298,0.0
3,std,0.018792,0.018792,0.0,0.007765,0.009987,0.011940,0.038539,3.5,118.5,...,0.193878,0.0,0.007795,0.035913,0.031459,1.359740e-16,0.005638,0.088282,0.141681,0.0


<a id="references"><a/>

# References

* [1] In-depth comparison of deep artificial neural network architectures on seismic events classification
* [2] Llaima volcano dataset: In-depth comparison of deep artificial neural network architectures on seismic events classification